## Simple Class


In [2]:
from functools import partial
class Counter:
    """Counter class"""
    
    def __init__(self, initial_count=0):
        self.count = initial_count
        
    def get(self):
        return self.count
    
    def increment(self):
        self.count += 1
    
c = Counter(initial_count=91)
c.increment()
print(c.get())

92


## Attributes

In [3]:
class Counter:
    all_counters = [] # class attribute
    
    def __init__(self, initial_count=0):
        Counter.all_counters.append(self)
        # no explicit field declaration
        self.count = initial_count
        
c1 = Counter(92)
c2 = Counter(62)
assert len(Counter.all_counters) == 2
assert c1.all_counters is c2.all_counters


 
## __dict__

In [4]:
c = Counter(92)
print(c.__class__)

<class '__main__.Counter'>


In [5]:
print(c.__dict__)

{'count': 92}


In [6]:
c.count == c.__dict__["count"]

True

In [7]:
c.__dict__["foo"] = 62
print(c.foo)

62


In [8]:
del c.foo
del c.__dict__["count"] # ~= .pop("count")
print(vars(c))

{}


## Class is an object


In [9]:
(Counter.__name__, Counter.__doc__, Counter.__module__)

('Counter', None, '__main__')

In [10]:
print(Counter.__bases__)
Counter.__dict__


(<class 'object'>,)


mappingproxy({'__module__': '__main__',
              'all_counters': [<__main__.Counter at 0x1c67ec39048>,
              '__init__': <function __main__.Counter.__init__(self, initial_count=0)>,
              '__dict__': <attribute '__dict__' of 'Counter' objects>,
              '__weakref__': <attribute '__weakref__' of 'Counter' objects>,
              '__doc__': None})

## Class is a statement

dict is a result of class body.

In [11]:
class Weird:
    f1, f2 = 0, 1
    for _ in range(10):
        f1, f2 = f2, f1 + f2

print(Weird.f1)

55


### attribute search

In [12]:
class A:
    x = 92
    
a = A()
print(vars(A))
print(vars(a))

print(a.x)

a.x = 62
print(vars(a))
print(a.x)
print(A.x)

{'__module__': '__main__', 'x': 92, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
{}
92
{'x': 62}
62
92


## Bound methods

In [13]:
class A:
    def foo(self):
        pass
a = A()
print(a.foo)
print(A.foo)
print(a.foo is A.foo)

<bound method A.foo of <__main__.A object at 0x000001C67EC28388>>
<function A.foo at 0x000001C67EC355E8>
False


In [14]:
from functools import partial
print(a.foo())
# same as
print(A.foo(a))

# same as
f = a.foo
g = partial(A.foo, a)
print(f())

print(g())



None
None
None
None


**obj.foo(bar)**
is same as
**obj.__class__.foo(obj, bar)**

## Properties

In [15]:
class Counter:
    def __init__(self, initial_count=0):
        self.count = initial_count

    def increment(self):
        self.count += 1

    @property
    def is_zero(self):
        return  self.count == 0

c = Counter()

assert c.is_zero #no ()

c.increment()

assert not c.is_zero


        

In [17]:
class Temperature:
    def __init__(self, *, celsius=0):
        self.celsius = celsius

    @property
    def fahrenheit(self):
        return self.celsius * 9 / 5 + 32

    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius= (value - 32) * 5 / 9

    @fahrenheit.deleter
    def fahrenheit(self):
        del self.celsius\

c = Temperature()
c.fahrenheit = 451

assert  c.celsius == 232.777777777777777

## __slots__

In [18]:
class A:
    __slots__ = ["x", "y"] # save memory

a = A()
print(a.__dict__)

AttributeError: 'A' object has no attribute '__dict__'

In [19]:
a.x = 92


In [20]:
a.z = 92

AttributeError: 'A' object has no attribute 'z'

## visibility

all fields and methods are public

## But there are some conventions and mangling

In [24]:
class A:
    def __init__(self):
        self.pub = 92
        self._priv = 62
        self.__mangled = 42 ##  will be converted to  _A__mangled at the compilation to bytecode stage

In [28]:
a = A()

assert  a.pub == 92
assert a._priv == 62
assert a._A__mangled == 42

## Inheritance


In [31]:
class Counter:
    def __init__(self, initial_count=0):
        self.count = initial_count

    def get(self):
        return self.count


class SquaredCounter(Counter): # may be multiple
    def get(self):  # can override
        return super().get() ** 2

c = SquaredCounter(91)
assert c.get() == 8281

In [32]:
assert isinstance(c, Counter)
assert issubclass(SquaredCounter, Counter)
assert issubclass(Counter, (str,object))

In [34]:
class A:
    def f(self):
        print("A")

class B:
    def f(self):
        print("B")

class C(A,B):
   pass

C().f()

A


In [36]:
class Base:
    def f(self):
        print("Base")

class A(Base):
    def f(self):
        print("A")
        super().f() #super is dynamic

class B(Base):
    def f(self):
        print("B")
        super().f()

class C(A, B):
    pass

C().f()


A
B
Base


In [37]:
assert C.mro() == [C, A, B, Base, object] # Method resolution order

#Super takes next class from mro

class A:
    def foo(self):
        super().foo()
        #is same as
        super(A, self).foo()

## Mixin

In [53]:
class Counter:
    def __init__(self, initial_count=0):
        self.count = initial_count
    def increment(self):
        self.count+=1


class DoublingMixing: #!!!
    def increment(self):
        super().increment()
        super().increment()

class DoublingCounter(DoublingMixing, Counter):
    pass

c = DoublingCounter()

assert c.count == 0
c.increment()
assert c.count == 2

## Decorators
same through decorator

In [56]:
import functools
def doubling(cls):
    orig_increment = cls.increment

    @functools.wraps(orig_increment)
    def increment(self):
        orig_increment(self)
        orig_increment(self)
    cls.increment = increment
    return cls

@doubling
class DoublingCounter(Counter):
    pass


c = DoublingCounter()

assert  c.count == 0
c.increment()
assert c.count == 2


## Magic methods


In [58]:
class Counter:
    def __init__(self, initial_count):
        self.count = initial_count

    def __lt__(self, other):
        return self.count < other.count

    def __eq__(self, other):
        return self.count == other.count

c1 = Counter(62)
c2 = Counter(92)

assert c1 < c2
assert (62).__lt__(92)
#assert c2 >= c1 # there is no __ge__ defined

we can use functools total_ordering decorator (it can define all 8 magic functions based on lt and gt

In [60]:
class Counter:
    def __init__(self, initial_count):
        self.count = initial_count

    def __repr__(self):
        return "Counter ({})".format(self.count)

    def __str__(self):
        return "Counted to {}".format(self.count)

c = Counter(92)
assert str(c) == f"{c}" == "Counted to 92"
assert repr(c) == f"{c!r}" == "Counter (92)"







In [61]:
class Counter:
    def __init__(self, initial_count):
        self.count = initial_count

    def __format__(self, format_spec):
        if format_spec == "bold":
            return f"**{self.count}**"
        return str(self.count)

c = Counter(92)

assert f"{c:bold}" == "**92**"

In [63]:
class Counter:
    def __init__(self, initial_count):
        self.count = initial_count

    def __hash__(self):
        # NB: a == b => hash(a) == hash(b)
        return hash(self.count)

    def __eq__(self, other):
        return self.count == other.count

assert len({Counter(92), Counter(92)}) == 1


In [66]:
class Counter:
    def __init__(self, initial_count):
        self.count = initial_count

    def __bool__(self):
        return self.count > 0

c = Counter(0)

if not c:
    print("empty")

empty


In [67]:

class Counter:
    def __init__(self, initial_count):
        self.count = initial_count

    def __add__(self, other):
        if not isinstance(other, int):
            return NotImplemented
        return Counter(self.count + other)

    def __radd__(self, other):
        return self + other

    def __bool__(self):
        return self.count > 0

c = Counter(0)

assert (c + 1).count == 1
assert (1 + c).count == 1

In [73]:
class Identity:
    def __call__(self, x):
        return x

assert Identity()(92) == 92

In [76]:
class n_times:
    def __init__(self, n):
        self.n = n

    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            for _ in range(self.n):
                func(*args, **kwargs)
        return inner()

In [77]:
c = Counter(10)
c = Counter.__call__(10)
c = type(Counter).__call__(Counter, 10)
c.count

10

In [78]:
import collections
class Silly:
    def __init__(self):
        self.__dict__['data'] = collections.defaultdict(lambda: 42)

    # calls this if there is no "item" in dict
    def __getattr__(self, item):
        return  self.data[item] #this works!

    def __setattr__(self, key, value):
        value = 42
        self.__dict__[key] = value

    def __delattr__(self, item):
        self.data.pop(item, None)

s = Silly()
assert s.foo == 42
s.foo = 92
assert s.foo == 42
del s.foo


In [80]:
class EvenMoreSilly:
    def __init__(self):
        __dict__ = super().__getattribute__("__dict__")
        __dict__["data"] = collections.defaultdict(lambda: 42)

    def __getattribute__(self, item):
        data = super().__getattribute__("data")
        return data[item]

    def __setattr__(self, key, value):
        value = 42
        data = super().__getattribute__("data")
        data[key] = value


s = EvenMoreSilly()
assert s.foo == 42
s.foo = 92
assert s.foo == 42
